# Flow Free SAT Solver

Mostraremos paso a paso las construccion de las reglas necesarias para solucionar el famoso juego **Flow Free**. Utilizaremos logica proposicional y el SAT Solver DPLL para este fin. Los mapas que manejamos se encuentran en la carpeta *mapas/* y sus coordenadas se encuentran en **x** de izquierda a derecha y en **y** de arriba a abajo.

In [34]:
# Librerias
import pycosat
import turtle
# Modulo
import Logica

# Funciones

### Funciones de Lectura

In [35]:
def FlowRead(MAP_FILE):
    """ Lee las situaciones iniciales de los archivos de ejemplo

    Args:
        MAP_FILE : path de archivo .txt con la situacion inicial.
    Returns:
        MAP_MATRIX : lista con la matriz de juego.
    """
    MAP_FILE = open(MAP_FILE, "r")
    MAP_MATRIX = [j.strip() for j in MAP_FILE]
    MAP_FILE.close()
    return MAP_MATRIX


def defineMap(matriz):
    """ Guarda la posicion de las terminales

    Args:
        matriz : lista con la matriz de juego
    Returns:
        dic : diccionario con las posiciones de las terminales
    """
    dic = {}
    dic_a = { 'R': 0, 'G': 1, 'B': 2, 'O': 3, 'P': 4 }

    for y in range(len(matriz)):
        for x in range(len(matriz[y])):
            if matriz[y][x] in ['R', 'G', 'B', 'O', 'P']:
                dic[(x, y)] = dic_a[matriz[y][x]]
    return dic

### Funciones para solucionar el juego (pycosat)

In [36]:
def lit_numero(l):
    """ Convierte un literal logico en un numero. Si contiene
    una negacion, el numero es negativo.
    """
    if '-' in l:
        return -ord(l[1:]) + 256
    else:
        return ord(l) - 256


def clausula_numero(C):
    """ Convierte a numero una lista de literales dentro de una clausula
    """
    return [lit_numero(l) for l in C]


def fnc_numero(S):
    """ Convierte a numero una lista de clausulas
    """
    return [clausula_numero(C) for C in S]


def obtener_int(mod):
    """ Transforma de entero a caracter
    """
    return {chr(256 + abs(n)): n > 0 for n in mod}


def resolver(formula):
    S = Logica.tseitin(formula)  # Conversion de la formula a forma clausal
    S = fnc_numero(S)  # Conversion de la forma clausal a numeros
    solution = pycosat.solve(S)  # Hallar solucion con DPLL  
    if solution == "UNSAT":  # Si es no satisfacible retorna None
        print(solution)
        return None
    # Si es satisfacible
    II = obtener_int(solution)  # Convierte enteros a caracteres
    lis = []
    for k in II:
        if (ord(k) >= OenCasilla.rango[0]) and (ord(k) <= OenCasilla.rango[1]) and II[k]:
            lis.append(k)
    return lis

### Funciones para visualizar la solucion

In [37]:
def coors(x, y):
    """ Transforma las posiciones de las celdas en coordenadas para Turtle
    """
    return -(((100 * Nx) / 2) - 50) + ( x * 100), ((( 100 * Ny) / 2) - 50) - (y * 100)


def visualizar(I):
    """ Visualizacion del mapa con la solucion
    """
    FlowWindow = turtle.Screen()
    FlowWindow.setup(100*Nx, 100*Ny)
    FlowWindow.title("FlowFree")
    FlowWindow.tracer(0)
    for T in range(Nc):
        FlowWindow.addshape('img/{0}.gif'.format(T))
    for d in D:
        for c in C:
            FlowWindow.addshape('img/{0}{1}.gif'.format(d, c))
    cell = turtle.Turtle()
    cell.pu()
    cell.ht()
    for t in pos_t.keys():
        cell.setpos(coors(t[0], t[1]))
        cell.shape('img/{}.gif'.format(pos_t[t]))
        cell.stamp()
    for valor in I:
        corx, cory, color, dirr = OenCasilla.inv(valor)
        cell.setpos(coors(corx, cory))
        cell.shape('img/{0}{1}.gif'.format(dirr, color))
        cell.stamp()
    FlowWindow.exitonclick()


# Situacion inicial

Utilizaremos la situacion inicial que se encuentra en _mapas/5x5/2.txt_ para analizar regla por regla

In [38]:
mapa = FlowRead("mapas/5x5/1.txt")  # Lectura del archivo

Nx = len(mapa[0])  # Casillas X
Ny = len(mapa)  # Casillas Y
Nc = 5  # Numero de colores posibles
Nd = 4  # Numero de direcciones posibles

# Listas
X = list(range(Nx))
Y = list(range(Ny))
C = list(range(Nc))
D = list(range(Nd))

# Diccionario con la transformacion de numero a caracter
# de los colores y direcciones posibles
NColores = { 'R': 0, 'G': 1, 'B': 2, 'O': 3, 'P': 4 }
Colores = { 0: 'R', 1: 'G', 2: 'B', 3: 'O', 4: 'P' }
Direcciones = { 0: 't', 1: 'b', 2: 'l', 3: 'r' }
direcciones_posibles = { 
    (0, 1): 'tb', (0, 2): 'tl', (0, 3): 'tr', 
    (1, 2): 'bl', (1, 3): 'br', (2, 3): 'lr' }

# Descriptor de las situacion
OenCasilla = Logica.Descriptor([Nx, Ny, Nc, Nd], chrInit=257)
pos_t = defineMap(mapa)  # Lista de las posiciones con terminales

# Regla 1

Asigna un color a todas las casillas si esta no es terminal.

In [39]:
def regla_1():
    Y_xy = []
    for x in X:
        for y in Y:
            if (x, y) not in pos_t:
                O_c = []
                for c in C:
                    nq = [OenCasilla.P([x, y, nc, d])
                          for nc in C if nc != c for d in D]
                    q = [OenCasilla.P([x, y, c, d]) for d in D]
                    formula = "("+Logica.Otoria(q)+"Y-"+Logica.Otoria(nq)+")"
                    O_c.append(formula)
                Y_xy.append(Logica.Otoria(O_c))
    return Logica.Ytoria(Y_xy)

In [41]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_1.png" width="250">

**Tiempo en encontrar una solucion:** < 1s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.* 

Se observa que todas las casillas tienen color violeta.

# Regla 2

Cada casilla debe tener dos direcciones si esta no es una terminal.

In [42]:
def regla_2():
    Y_xy = []
    for x in X:
        for y in Y:
            if (x, y) not in pos_t.keys():
                O_d = []
                for d in direcciones_posibles.keys():
                    pq = [Logica.Ytoria(
                        [OenCasilla.P([x, y, c, d[0]]), OenCasilla.P([x, y, c, d[1]])]) for c in C]
                    npq = [Logica.Ytoria([OenCasilla.P([x, y, c, u[0]]), OenCasilla.P(
                        [x, y, c, u[1]])]) for c in C for u in direcciones_posibles.keys() if u != d]
                    pq = "("+Logica.Otoria(pq)+"Y-"+Logica.Otoria(npq)+")"
                    O_d.append(pq)
                Y_xy.append(Logica.Otoria(O_d))
    return Logica.Ytoria(Y_xy)

In [43]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_2.png" width="250">

**Tiempo en encontrar una solucion:** < 5s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

Se observa que todas las casillas tienen direcciones *top-bottom* de color rojo.

# Regla 3

Asigna que las terminales solo pueden tener un vecino apuntando a su direccion, el cual debe tener el mismo color que la terminal.

In [44]:
def vectort(T, vecino_escogido):
    if vecino_escogido[0] == T[0] + 1:
        return 2
    elif vecino_escogido[0] == T[0] - 1:
        return 3
    elif vecino_escogido[1] == T[1] + 1:
        return 0
    elif vecino_escogido[1] == T[1] - 1:
        return 1


def regla_3():
    Y_pos_t = []
    for T in pos_t.keys():
        # Encuentra las casillas que son vecinos de la terminal y que no son terminal
        vecinos = [(x, y) for x in X for y in Y if ((x, y) not in pos_t.keys()) and (
            ((x+1 == T[0] or x-1 == T[0]) and y == T[1]) or ((y+1 == T[1] or y-1 == T[1]) and x == T[0]))]
        if len(vecinos) == 1:
        # Si solo tiene una casilla vecina que no es terminal esta debe tener una 
        # direccion que apunte a la terminal
            Y_pos_t.append(OenCasilla.P(
                [vecinos[0][0], vecinos[0][1], pos_t[T], vectort(T, vecinos[0])]))
            continue
        O_par_t = []
        for vecino_escogido in vecinos:
        # Solo uno de los posibles vecinos puede estar apuntando a su direccion
            otros_vecinos = [i for i in vecinos if i != vecino_escogido]
            vtp = OenCasilla.P(
                [vecino_escogido[0], vecino_escogido[1], pos_t[T], vectort(T, vecino_escogido)])
            ovtotal = []
            for ov in otros_vecinos:
                ovc = [OenCasilla.P(
                    [ov[0], ov[1], c, vectort(T, (ov[0], ov[1]))])for c in C]
                ovtotal.append(Logica.Otoria(ovc))
            formula = "("+vtp+"Y-"+Logica.Otoria(ovtotal)+")"
            O_par_t.append(formula)
        Y_pos_t.append(Logica.Otoria(O_par_t))
    return Logica.Ytoria(Y_pos_t)

In [45]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_3.png" width="250">

**Tiempo en encontrar una solucion:** < 5s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

Se observa que todas las terminales tienen exactamente un vecino de su mismo color apuntando en su direccion. Ademas de cumplirse las reglas anteriores.

# Regla 10

Regla de ajuste:
1. Las casillas de la primera fila no pueden tener direccion *top*.
2. Las casillas de la ultima fila no pueden tener direccion *bottom*.
3. Las casillas de la primera columna no pueden tener direccion *left*.
4. Las casillas de la primera columna no pueden tener direcicon *right*.

In [46]:
def regla_10():
    Y_xy = []
    for x in X:
        for y in Y:
            if (x, y) not in pos_t.keys():
                if x == 0:
                    f = "-" + \
                        Logica.Otoria([OenCasilla.P([x, y, c, 2]) for c in C])
                    Y_xy.append(f)
                elif x == Nx-1:
                    f = "-" + \
                        Logica.Otoria([OenCasilla.P([x, y, c, 3]) for c in C])
                    Y_xy.append(f)
                if y == 0:
                    f = "-" + \
                        Logica.Otoria([OenCasilla.P([x, y, c, 0]) for c in C])
                    Y_xy.append(f)
                elif y == Ny-1:
                    f = "-" + \
                        Logica.Otoria([OenCasilla.P([x, y, c, 1]) for c in C])
                    Y_xy.append(f)
    return Logica.Ytoria(Y_xy)

In [47]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT.append(regla_10())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_10.png" width="250">

**Tiempo en encontrar una solucion:** < 6s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

Se observa que todas las casillas que se encuentran en los extremos del mapa cumplen las condiciones impuestas por la regla. Comparandolo con la solucion anterior, por ejemplo, vemos que las casillas de la primera y ultima columna no tienen direccion *top* y *bottom*, respectivamente.

# Reglas de direcciones

Estas reglas tienen la caracteristiza de no poder visualizarse de forma adecuada agregandolas de una en una, ya que es posible que una de las direcciones posibles de las que este tratando la regla no aparezca en la solucion.

Por esto ultimo, **implementaremos las reglas en desorden para observar mejor el efecto**.

### Regla 4

Regla para la cuando una casilla tiene las direcciones *left-right*:
- Su vecino superior no debe tener *bottom*.
- Su vecino derecho debe tener *left*.
- Su vecino inferior no debe tener *top*.
- Su vecino izquierdo debe tener *right*.

In [48]:
# Left-Right en numeros son la direccion 2-3
def regla_4():
    Y_xy = []
    for x in X:
        for y in Y:
            if (x,y) not in pos_t.keys() and (x!=0 and x!=Nx-1):
                O_c = []
                for c in C:
                    formula = "("+OenCasilla.P([x,y,c,2])+"Y"+OenCasilla.P([x,y,c,3])+")"
                    vecinos = []
                    if((x+1,y) in pos_t.keys()):
                        if ((x-1,y) in pos_t.keys()):
                            continue
                        else:
                            vecinos.append( OenCasilla.P([x - 1, y, pos_t[(x + 1, y)], 3]) )
                    elif ((x-1,y) in pos_t.keys()):
                        vecinos.append( OenCasilla.P([x + 1, y, pos_t[(x-1, y)], 2]) )
                    else:
                        vecinos.append( "("+OenCasilla.P([x + 1, y, c ,2]) + "Y" + OenCasilla.P([x - 1,y,c,3])+")")
                    opuestos_a = ["-"+OenCasilla.P([x,y+1,o,0]) for o in C if y+1 in Y]
                    opuestos_b = ["-"+OenCasilla.P([x,y-1,o,1]) for o in C if y-1 in Y]
                    total_op = Logica.Ytoria(vecinos + opuestos_a + opuestos_b)
                    formula_total = "("+formula+">"+total_op+")"
                    O_c.append(formula_total)
                if len(O_c) != 0:
                    Y_xy.append(Logica.Ytoria(O_c))
    return Logica.Ytoria(Y_xy)

In [49]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT.append(regla_10())
    SAT.append(regla_4())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_4.png" width="250">

**Tiempo en encontrar una solucion:** < 6s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

La solucion anterior incumplia esta regla en la celda (3, 0) en la que habia una casilla con direcciones *left-right* cuyos vecinos izquierdo y derecho no se encontraban conectadas con esta.

En la nueva solucion que encontro, observamos como siempre que una casilla tiene direcciones *left-right* sus vecinos se encuentran conectadas con estas de alguna manera.

### Regla 6

Regla para la cuando una casilla tiene las direcciones *top-left*:
- Su vecino superior debe tener *bottom*.
- Su vecino derecho no debe tener *left*.
- Su vecino inferior no debe tener *top*.
- Su vecino izquierdo debe tener *right*.

In [50]:
# Top-Left en numeros son la direccion 0-2
def regla_6():  
    Y_xy = []
    for x in X:
        for y in Y:
            if (x, y) not in pos_t.keys() and (y != 0 and x != 0):
                O_c = []
                for c in C:
                    formula = "(" + OenCasilla.P([x, y, c, 0]) + \
                        "Y" + OenCasilla.P([x, y, c, 2]) + ")"
                    vecinos = []
                    if((x - 1, y) in pos_t.keys()):
                        if ((x, y - 1) in pos_t.keys()):
                            continue
                        else:
                            vecinos.append(OenCasilla.P(
                                [x, y - 1, pos_t[(x - 1, y)], 1]))
                    elif ((x, y - 1) in pos_t.keys()):
                        vecinos.append(OenCasilla.P(
                            [x - 1, y, pos_t[(x, y - 1)], 3]))
                    else:
                        vecinos.append(
                            "(" + OenCasilla.P([x - 1, y, c, 3]) + "Y" + OenCasilla.P([x, y - 1, c, 1]) + ")")
                    opuestos_a = [
                        "-" + OenCasilla.P([x, y + 1, o, 0]) for o in C if y + 1 in Y]
                    opuestos_b = [
                        "-" + OenCasilla.P([x + 1, y, o, 2]) for o in C if x + 1 in X]
                    total_op = Logica.Ytoria(vecinos + opuestos_a + opuestos_b)
                    formula_total = "(" + formula + ">" + total_op + ")"
                    O_c.append(formula_total)
                if len(O_c) != 0:
                    Y_xy.append(Logica.Ytoria(O_c))
    return Logica.Ytoria(Y_xy)

In [51]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT.append(regla_10())
    SAT.append(regla_4())
    SAT.append(regla_6())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_6.png" width="250">

**Tiempo en encontrar una solucion:** < 7s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

La solucion anterior incumplia esta regla en la celda (1, 4) en la que habia una casilla azul con direcciones *top-left* cuyo vecino izquierdo era verde.

En la nueva solucion que encontro, observamos que la unica celda que cambio es la (1, 4) hacia la direccion *top-right* donde todavia falta implementar una regla.

### Regla 5

Regla para la cuando una casilla tiene las direcciones *top-right*: 
- Su vecino superior debe tener *bottom*.
- Su vecino derecho debe tener *left*.
- Su vecino inferior no debe tener *top*.
- Su vecino izquierdo no debe tener *right*.

In [52]:
# Top-Right en numeros son la direccion 0-3
def regla_5():
    Y_xy = []
    for x in X:
        for y in Y:
            if (x, y) not in pos_t.keys() and (y != 0 and x != Nx-1):
                O_c = []
                for c in C:
                    formula = "(" + OenCasilla.P([x, y, c, 0]) + \
                        "Y" + OenCasilla.P([x, y, c, 3]) + ")"
                    vecinos = []
                    if((x + 1, y) in pos_t.keys()):
                        if ((x, y - 1) in pos_t.keys()):
                            continue
                        else:
                            vecinos.append(OenCasilla.P(
                                [x, y - 1, pos_t[(x + 1, y)], 1]))
                    elif ((x, y - 1) in pos_t.keys()):
                        vecinos.append(OenCasilla.P(
                            [x + 1, y, pos_t[(x, y - 1)], 2]))
                    else:
                        vecinos.append(
                            "(" + OenCasilla.P([x + 1, y, c, 2]) + "Y" + OenCasilla.P([x, y - 1, c, 1]) + ")")
                    opuestos_a = [
                        "-" + OenCasilla.P([x, y + 1, o, 0]) for o in C if y + 1 in Y]
                    opuestos_b = [
                        "-" + OenCasilla.P([x - 1, y, o, 3]) for o in C if x - 1 in X]
                    total_op = Logica.Ytoria(vecinos + opuestos_a + opuestos_b)
                    formula_total = "(" + formula + ">" + total_op + ")"
                    O_c.append(formula_total)
                if len(O_c) != 0:
                    Y_xy.append(Logica.Ytoria(O_c))
    return Logica.Ytoria(Y_xy)

In [53]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT.append(regla_10())
    SAT.append(regla_4())
    SAT.append(regla_6())
    SAT.append(regla_5())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_5.png" width="250">

**Tiempo en encontrar una solucion:** < 7s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

La solucion anterior incumplia esta regla en la celda (1, 4) en la que habia una casilla azul con direcciones *top-right* cuyo vecino derecho era verde y no se encontraba conectado.

En la nueva solucion que encontro, observamos como cambia toda la ultima fila. Gracias a esta regla vemos que la casilla (1, 4) debe ser verde con direcciones *left-right*.

### Regla 7

Regla para la cuando una casilla tiene las direcciones *top-bottom*:
- Su vecino superior debe tener *bottom*.
- Su vecino derecho no debe tener *left*.
- Su vecino inferior debe tener *top*.
- Su vecino izquierdo no debe tener *right*.

In [54]:
# Top-Bottom en numeros es 0-1
def regla_7():
    Y_xy = []
    for x in X:
        for y in Y:
            if (x,y) not in pos_t.keys() and (y!=0 and y!=Ny-1):
                O_c = []
                for c in C:
                    formula = "(" + OenCasilla.P([x, y, c, 0]) + "Y" + OenCasilla.P([x, y, c, 1]) + ")"
                    vecinos = []
                    if((x, y + 1) in pos_t.keys()):
                        if ((x, y - 1) in pos_t.keys()): # bottom es Terminal y top es Terminal
                            continue
                        else: # bottom es Terminal y top no es Terminal
                            vecinos.append( OenCasilla.P([x, y - 1, pos_t[(x, y + 1)], 1]) )
                    elif ((x, y - 1) in pos_t.keys()): # bottom no es Terminal y top es Terminal
                        vecinos.append( OenCasilla.P([x, y + 1, pos_t[(x, y - 1)], 0]) )
                    else: # ni bottom ni top son Terminal
                        vecinos.append( "(" + OenCasilla.P([x, y + 1, c , 0]) + "Y" + OenCasilla.P([x ,y - 1, c, 1]) + ")")
                    opuestos_a = ["-" + OenCasilla.P([x - 1, y, o, 3]) for o in C if x - 1 in X]
                    opuestos_b = ["-" + OenCasilla.P([x + 1, y, o, 2]) for o in C if x + 1 in X]
                    total_op = Logica.Ytoria(vecinos + opuestos_a + opuestos_b)
                    formula_total = "(" + formula + ">" + total_op + ")"
                    O_c.append(formula_total)
                if len(O_c) != 0:
                    Y_xy.append(Logica.Ytoria(O_c))
    return Logica.Ytoria(Y_xy)

In [55]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT.append(regla_10())
    SAT.append(regla_4())
    SAT.append(regla_6())
    SAT.append(regla_5())
    SAT.append(regla_7())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_7.png" width="250">

**Tiempo en encontrar una solucion:** < 8s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

La solucion anterior incumplia esta regla en la celda (2, 2) en la que habia una casilla azul con direcciones *top-bottom* cuyo vecino superior era rojo y no se encontraba conectado.

En la nueva solucion que encontro, observamos como cambia la casilla (2, 2) logrando obtener una solucion correcta para el juego. **Sin embargo, existen juegos que todavia no se pueden solucionar**.

## Cambio de situacion inicial

El mapa que se encuentra en *mapas/5x5/5.txt* no encuentra una solucion correcta con las reglas que hemos definido hasta ahora.

In [56]:
mapa = FlowRead("mapas/5x5/5.txt")  # Lectura del archivo

Nx = len(mapa[0])  # Casillas X
Ny = len(mapa)  # Casillas Y
Nc = 5  # Numero de colores posibles
Nd = 4  # Numero de direcciones posibles

# Listas
X = list(range(Nx))
Y = list(range(Ny))
C = list(range(Nc))
D = list(range(Nd))

# Diccionario con la transformacion de numero a caracter
# de los colores y direcciones posibles
NColores = {'R': 0, 'G': 1, 'B': 2, 'O': 3, 'P': 4}
Colores = {0: 'R', 1: 'G', 2: 'B', 3: 'O', 4: 'P'}
Direcciones = {0: 't', 1: 'b', 2: 'l', 3: 'r'}
direcciones_posibles = {
    (0, 1): 'tb', (0, 2): 'tl', (0, 3): 'tr',
    (1, 2): 'bl', (1, 3): 'br', (2, 3): 'lr'}

# Descriptor de las situacion
OenCasilla = Logica.Descriptor([Nx, Ny, Nc, Nd], chrInit=257)
pos_t = defineMap(mapa)  # Lista de las posiciones con terminales

In [57]:
M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/cambio_situacion.png" width="250">

Observamos como no todos las casillas se encuentran correctamente conectadas.

### Regla 8

Regla para la cuando una casilla tiene las direcciones *left-bottom*:
- Su vecino superior no debe tener *bottom*.
- Su vecino derecho no debe tener *left*.
- Su vecino inferior debe tener *top*.
- Su vecino izquierdo debe tener *right*.

In [58]:
# Left-Bottom en numeros son 2-1
def regla_8():
    Y_xy = []
    for x in X:
        for y in Y:
            if (x, y) not in pos_t.keys() and (x != 0 and y != Ny-1):
                O_c = []
                for c in C:
                    formula = "(" + OenCasilla.P([x, y, c, 2]) + \
                        "Y" + OenCasilla.P([x, y, c, 1]) + ")"
                    vecinos = []
                    if((x, y + 1) in pos_t.keys()):
                        if ((x - 1, y) in pos_t.keys()):  # bottom es Terminal y left es Terminal
                            continue
                        else:  # bottom es Terminal y left no es Terminal
                            vecinos.append(OenCasilla.P(
                                [x - 1, y, pos_t[(x, y + 1)], 3]))
                    elif ((x - 1, y) in pos_t.keys()):  # bottom no es Terminal y left es Terminal
                        vecinos.append(OenCasilla.P(
                            [x, y + 1, pos_t[(x - 1, y)], 0]))
                    else:  # ni bottom ni left son Terminal
                        vecinos.append(
                            "(" + OenCasilla.P([x, y + 1, c, 0]) + "Y" + OenCasilla.P([x - 1, y, c, 3]) + ")")
                    opuestos_a = [
                        "-" + OenCasilla.P([x, y - 1, o, 1]) for o in C if y - 1 in Y]
                    opuestos_b = [
                        "-" + OenCasilla.P([x + 1, y, o, 2]) for o in C if x + 1 in X]
                    total_op = Logica.Ytoria(vecinos + opuestos_a + opuestos_b)
                    formula_total = "(" + formula + ">" + total_op + ")"
                    O_c.append(formula_total)
                if len(O_c) != 0:
                    Y_xy.append(Logica.Ytoria(O_c))
    return Logica.Ytoria(Y_xy)

In [59]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT.append(regla_10())
    SAT.append(regla_4())
    SAT.append(regla_6())
    SAT.append(regla_5())
    SAT.append(regla_7())
    SAT.append(regla_8())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_8.png" width="250">

**Tiempo en encontrar una solucion:** < 9s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

La solucion anterior incumplia esta regla en las casilla azul (1, 0) que tenia direcciones *left-bottom* cuyo vecino izquierdo era de color rojo.

En la nueva solucion que encontro, observamos que esta casilla (1, 0) cambia de direcciones a *right-bottom* que es la regla que aun nos falta por definir.

### Regla 9

Regla para la cuando una casilla tiene las direcciones *right-bottom*:
- Su vecino superior no debe tener *bottom*.
- Su vecino derecho debe tener *left*.
- Su vecino inferior debe tener *top*.
- Su vecino izquierdo no debe tener *right*.

In [60]:
# Right-Bottom en numeros es 3-1
def regla_9():
    Y_xy = []
    for x in X:
        for y in Y:
            if (x, y) not in pos_t.keys() and (x != Nx-1 and y != Ny-1):
                O_c = []
                for c in C:
                    formula = "(" + OenCasilla.P([x, y, c, 3]) + \
                        "Y" + OenCasilla.P([x, y, c, 1]) + ")"
                    vecinos = []
                    if((x, y + 1) in pos_t.keys()):
                        if ((x + 1, y) in pos_t.keys()):  # bottom es Terminal y right es Terminal
                            continue
                        else:  # bottom es Terminal y right no es Terminal
                            vecinos.append(OenCasilla.P(
                                [x + 1, y, pos_t[(x, y + 1)], 2]))
                    elif ((x + 1, y) in pos_t.keys()):  # bottom no es Terminal y right es Terminal
                        vecinos.append(OenCasilla.P(
                            [x, y + 1, pos_t[(x + 1, y)], 0]))
                    else:  # ni bottom ni right son Terminal
                        vecinos.append(
                            "(" + OenCasilla.P([x, y + 1, c, 0]) + "Y" + OenCasilla.P([x + 1, y, c, 2]) + ")")
                    opuestos_a = [
                        "-" + OenCasilla.P([x, y - 1, o, 1]) for o in C if y - 1 in X]
                    opuestos_b = [
                        "-" + OenCasilla.P([x - 1, y, o, 3]) for o in C if x - 1 in X]
                    total_op = Logica.Ytoria(vecinos + opuestos_a + opuestos_b)
                    formula_total = "(" + formula + ">" + total_op + ")"
                    O_c.append(formula_total)
                if len(O_c) != 0:
                    Y_xy.append(Logica.Ytoria(O_c))
    return Logica.Ytoria(Y_xy)


In [61]:
def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT.append(regla_10())
    SAT.append(regla_4())
    SAT.append(regla_6())
    SAT.append(regla_5())
    SAT.append(regla_7())
    SAT.append(regla_8())
    SAT.append(regla_9())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)

M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)

<img src="example_imgs_notebook/regla_9.png" width="250">

**Tiempo en encontrar una solucion:** < 11s. 

*La solucion aparece en una pestaña nueva ya que estamos utilizando Turtle.*

La solucion anterior incumplia esta regla en las casilla azul (0, 0) que tenia direcciones *right-bottom* cuyo vecino derecho era de color azul y no se encontraba conectada.

Vemos como en la nueva solucion, se corrigen todas las casillas que tenian direcciones *right-bottom* y se encuentra la solucion correcta.

# Experimentacion

Invitamos al lector que experimente con los demas ejemplos que se encuentran en la carpeta *mapas/* utilizando el siguiente codigo:

In [ ]:
path_mapa = "mapas/6x6/1.txt"

In [ ]:
mapa = FlowRead(path_mapa)  # Lectura del archivo

Nx = len(mapa[0])  # Casillas X
Ny = len(mapa)  # Casillas Y
Nc = 5  # Numero de colores posibles
Nd = 4  # Numero de direcciones posibles

# Listas
X = list(range(Nx))
Y = list(range(Ny))
C = list(range(Nc))
D = list(range(Nd))

# Diccionario con la transformacion de numero a caracter
# de los colores y direcciones posibles
NColores = {'R': 0, 'G': 1, 'B': 2, 'O': 3, 'P': 4}
Colores = {0: 'R', 1: 'G', 2: 'B', 3: 'O', 4: 'P'}
Direcciones = {0: 't', 1: 'b', 2: 'l', 3: 'r'}
direcciones_posibles = {
    (0, 1): 'tb', (0, 2): 'tl', (0, 3): 'tr',
    (1, 2): 'bl', (1, 3): 'br', (2, 3): 'lr'}

# Descriptor de las situacion
OenCasilla = Logica.Descriptor([Nx, Ny, Nc, Nd], chrInit=257)
pos_t = defineMap(mapa)  # Lista de las posiciones con terminales


def flowSAT():
    SAT = []
    SAT.append(regla_1())
    SAT.append(regla_2())
    SAT.append(regla_3())
    SAT.append(regla_10())
    SAT.append(regla_4())
    SAT.append(regla_6())
    SAT.append(regla_5())
    SAT.append(regla_7())
    SAT.append(regla_8())
    SAT.append(regla_9())
    SAT = list(filter(None, SAT))
    return Logica.Ytoria(SAT)


M = resolver(flowSAT())  # Resuelve
# Si encuentra una solucion, muestra una visualizacion
if M:
    visualizar(M)